# Advent of Code, 13th Dec 2024

## Claw Contraption - Part 1

Next up: the lobby of a resort on a tropical island. The Historians take a moment to admire the hexagonal floor tiles before spreading out.

Fortunately, it looks like the resort has a new arcade! Maybe you can win some prizes from the claw machines?

The claw machines here are a little unusual. Instead of a joystick or directional buttons to control the claw, these machines have two buttons labeled A and B. Worse, you can't just put in a token and play; it costs **3 tokens** to push the A button and **1 token** to push the B button.

With a little experimentation, you figure out that each machine's buttons are configured to move the claw a specific amount to the **right** (along the X axis) and a specific amount **forward** (along the Y axis) each time that button is pressed.

Each machine contains one **prize**; to win the prize, the claw must be positioned **exactly** above the prize on both the X and Y axes.

You wonder: what is the smallest number of tokens you would have to spend to win as many prizes as possible? You assemble a list of every machine's button behavior and prize location (your puzzle input). For example:

```
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400
```

```
Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176
```

```
Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450
```

```
Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279
```

This list describes the button configuration and prize location of four different claw machines.

For now, consider just the first claw machine in the list:

* Pushing the machine's A button would move the claw 94 units along the X axis and 34 units along the Y axis.
* Pushing the B button would move the claw 22 units along the X axis and 67 units along the Y axis.
* The prize is located at X=8400, Y=5400; this means that from the claw's initial position, it would need to move exactly 8400 units along the X axis and exactly 5400 units along the Y axis to be perfectly aligned with the prize in this machine.

The cheapest way to win the prize is by pushing the A button 80 times and the B button 40 times. This would line up the claw along the X axis (because 80\*94 + 40\*22 = 8400) and along the Y axis (because 80\*34 + 40\*67 = 5400). Doing this would cost 80\*3 tokens for the A presses and 40\*1 for the B presses, a total of **280** tokens.

For the second and fourth claw machines, there is no combination of A and B presses that will ever win a prize.

For the third claw machine, the cheapest way to win the prize is by pushing the A button 38 times and the B button 86 times. Doing this would cost a total of **200** tokens.

So, the most prizes you could possibly win is two; the minimum tokens you would have to spend to win all (two) prizes is **480**.

You estimate that each button would need to be pressed **no more than 100 times** to win a prize. How else would someone be expected to play?

Figure out how to win as many prizes as possible. **What is the fewest tokens you would have to spend to win all possible prizes?**

### Read in the data

In [1]:
#!F#
let inputFilePath = "aoc-2024-12-13-puzzle-input.txt"

Read the lines from the file.

In [2]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines.Length

1279

### Solve the problem

We define some types to capture the details of buttons and machines.

In [3]:
type Button = { deltaX: int; deltaY: int}

type Machine = { buttonA: Button; buttonB: Button; prizeX: int; prizeY: int }

Now we use regular expressions to read the lines from the input text, and construct our list of machines.

In [4]:
open System.Text.RegularExpressions

In [5]:
let buttonARegex = new Regex(@"^Button A: X\+(\d+), Y\+(\d+)$")
let buttonBRegex = new Regex(@"^Button B: X\+(\d+), Y\+(\d+)$")
let prizeRegex = new Regex(@"^Prize: X=(\d+), Y=(\d+)$")

let readButtons (lines: string list): Machine list =
    if (lines.IsEmpty)
    then []
    else
        let rec iterate (lines: string list) (machines: Machine list) (buttonA: Button option) (buttonB: Button option): Machine list =
            if (lines.IsEmpty)
            then machines
            else
                match lines[0] with
                | buttonAStr when buttonA.IsNone && buttonARegex.IsMatch(buttonAStr) ->
                    let buttonAMatch = buttonARegex.Matches(buttonAStr)[0]
                    iterate (lines.Tail) machines (Some {deltaX = buttonAMatch.Groups[1].Value |> Int32.Parse; deltaY = buttonAMatch.Groups[2].Value |> Int32.Parse}) buttonB
                | buttonBStr when buttonB.IsNone && buttonBRegex.IsMatch(buttonBStr) ->
                    let buttonBMatch = buttonBRegex.Matches(buttonBStr)[0]
                    iterate (lines.Tail) machines buttonA (Some {deltaX = buttonBMatch.Groups[1].Value |> Int32.Parse; deltaY = buttonBMatch.Groups[2].Value |> Int32.Parse})
                | prizeStr when prizeRegex.IsMatch(prizeStr) ->
                    let prizeMatch = prizeRegex.Matches(prizeStr)[0]
                    let machine = { buttonA = buttonA.Value; buttonB = buttonB.Value; prizeX = prizeMatch.Groups[1].Value |> Int32.Parse; prizeY = prizeMatch.Groups[2].Value |> Int32.Parse }  
                    iterate (lines.Tail) (machine::machines) None None
                | _ ->
                    iterate lines.Tail machines buttonA buttonB
        iterate lines [] None None |> List.rev

In [6]:
let machines = readButtons lines
machines.Length

320

In [7]:
machines[0]

{ buttonA = { deltaX = 26\n deltaY = 56 }\n buttonB = { deltaX = 43\n deltaY = 22 }\n prizeX = 6138\n prizeY = 6756 } buttonA { deltaX = 26\n deltaY = 56 } deltaX 26 deltaY 56 buttonB { deltaX = 43\n deltaY = 22 } deltaX 43 deltaY 22 prizeX 6138 prizeY 6756

So for each machine, we need to calculate what combinations of button A and button B presses get you to the prize location.  There could be *no solution*, or there could be *a single solution*.  For each machines, the equations that we have are:

* $N_A A_x + N_B B_x = P_x$
* $N_A A_y + N_B B_y = P_y$

where $N_A$ and $N_B$ must be non-negative integers.  We could solve this as a simple matrix problem, but it's probably simpler (in some ways) to just iterate through possible values for $N_A$ and $N_B$ and see which give valid solutions.

In [8]:
let solutions (machine: Machine): (int * int) list =
    (seq {
        for nA in 0..(max (machine.prizeX / machine.buttonA.deltaX) (machine.prizeY / machine.buttonA.deltaY)) do
            for nB in 0..(max (machine.prizeX / machine.buttonB.deltaX) (machine.prizeY / machine.buttonB.deltaY)) do
                if (nA * machine.buttonA.deltaX + nB * machine.buttonB.deltaX = machine.prizeX && nA * machine.buttonA.deltaY + nB * machine.buttonB.deltaY = machine.prizeY)
                then yield (nA, nB)
    } |> Seq.toList)

In [9]:
let machineSolutions =
    machines
    |> List.map (fun machine -> (machine, solutions machine))
    |> List.filter (fun (_,solutions) -> not solutions.IsEmpty)
    |> Map.ofList
machineSolutions.Count

129

So only 129 of the 320 machines have solutions.

In [10]:
let machineTokens =
    machineSolutions.Keys
    |> Seq.map (fun machine -> (machine, machineSolutions[machine] |> List.map (fun (nA, nB) -> nA*3 + nB)))
    |> Map.ofSeq

In [11]:
let minTokens = machineTokens.Values |> Seq.map List.min |> Seq.sum
minTokens

28138

## Claw Contraption - Part 2

As you go to win the first prize, you discover that the claw is nowhere near where you expected it would be. Due to a unit conversion error in your measurements, the position of every prize is actually 10000000000000 higher on both the X and Y axis!

Add 10000000000000 to the X and Y position of every prize. After making this change, the example above would now look like this:

```
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=10000000008400, Y=10000000005400
```

```
Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=10000000012748, Y=10000000012176
```

```
Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=10000000007870, Y=10000000006450
```

```
Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=10000000018641, Y=10000000010279
```

Now, it is only possible to win a prize on the second and fourth claw machines. Unfortunately, it will take **many more than 100 presses** to do so.

Using the corrected prize coordinates, figure out how to win as many prizes as possible. **What is the fewest tokens you would have to spend to win all possible prizes?**

### Solve the problem

Now we have to increase the prize positions by a large fixed offset.  This probably makes the 'Part 1' algorithm too slow, in which case it will make sense to use the matrix solution that we eschewed in Part 1, and to switch from integer values to BigDecimal values.

In [12]:
let offset = 10000000000000L

In [13]:
type Machine2 = { buttonA: Button; buttonB: Button; prize2X: int64; prize2Y: int64 }

In [14]:
let machines2 = machines |> List.map (fun machine -> { buttonA = machine.buttonA; buttonB = machine.buttonB; prize2X = (int64 machine.prizeX) + offset; prize2Y = (int64 machine.prizeY) + offset })
machines2.Length

320

It's quickly checked that the solution to

* $N_A A_x + N_B B_x = P_x$
* $N_A A_y + N_B B_y = P_y$

is, assuming $A_x B_y - B_x A_y \neq 0$,

* $N_A = \frac{B_y P_x - B_x P_y}{A_x B_y - B_x A_y}$
* $N_B = \frac{A_x P_y - A_y P_x}{A_x B_y - B_x A_y}$

These solutions are only valid for this problem, though, if $N_A$ and $N_B$ are both integers, not fractions.

In [15]:
open System.Numerics

let bigDiv (a: BigInteger) (b: BigInteger): (BigInteger*BigInteger) option =
    match b with
    | zero when zero = BigInteger.Zero -> None
    | _ -> Some (BigInteger.DivRem(a, b))

In [19]:
let solutions2 (machine: Machine2): (int64 * int64) list =
    let denom = (BigInteger machine.buttonA.deltaX) * (BigInteger machine.buttonB.deltaY) - (BigInteger machine.buttonA.deltaY) * (BigInteger machine.buttonB.deltaX)
    let nA = bigDiv ((BigInteger machine.buttonB.deltaY) * (BigInteger machine.prize2X) - (BigInteger machine.buttonB.deltaX) * (BigInteger machine.prize2Y)) denom
    let nB = bigDiv ((BigInteger machine.buttonA.deltaX) * (BigInteger machine.prize2Y) - (BigInteger machine.buttonA.deltaY) * (BigInteger machine.prize2X)) denom
    if (nA.IsSome && nB.IsSome && snd nA.Value = BigInteger.Zero && snd nB.Value = BigInteger.Zero)
    then [int64 (fst nA.Value), int64 (fst nB.Value)]
    else []

In [20]:
let machineSolutions2 =
    machines2
    |> List.map (fun machine -> (machine, solutions2 machine))
    |> List.filter (fun (_,solutions) -> not solutions.IsEmpty)
    |> Map.ofList
machineSolutions2.Count

191

In [22]:
let machineTokens2 =
    machineSolutions2.Keys
    |> Seq.map (fun machine -> (machine, machineSolutions2[machine] |> List.map (fun (nA, nB) -> nA*3L + nB)))
    |> Map.ofSeq

In [24]:
let minTokens2 = machineTokens2.Values |> Seq.map List.min |> Seq.sum
minTokens2

108394825772874